<h1>Separating JSON Data into Columns (Movies)</h1>

In [36]:
import csv, json, gensim, datetime, time
import nltk
import re
import gensim
import numpy as np
from gensim import corpora

rowslist = []
genreslist = []
productioncompanieslist = []
overviewlist = []

stop_list = nltk.corpus.stopwords.words('english')
stemmer = nltk.stem.porter.PorterStemmer()

def readJSON(row, rownumber, list, keyword):
    jsonstring = row[rownumber]

    jsonobj = json.loads(jsonstring)

    for jsonelement in jsonobj:
        keywordelement = jsonelement[keyword]       
        list.append(keywordelement.lower()) 
    
    return list
    
def processText(row, rownumber1, rownumber2, list):
    text = row[rownumber1].split(" ") + row[rownumber2].split(" ") #contains overview and tagline
    
    text1 = [w.lower() for w in text]
    text2 = [w for w in text1 if re.search('^[a-z]+$', w)]
    text3 = [w for w in text2 if w not in stop_list]
    text4 = [stemmer.stem(w) for w in text3]  
    
    list = text4
    
    return list
    
#read file
with open('tmdb_All_movies.csv', encoding='utf-8') as csv_file: #change accordingly
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        try:        
            if line_count == 0:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            elif line_count != 5:
                eachnewrow = []
                genres = []
                productioncompanies = []
                productioncountries = []
                overview = []

                eachnewrow.append(row[0]) #budget

                if row[1] != "":
                    #handle json genres
                    genres = readJSON(row, 1, genres, 'name')

                eachnewrow.append(genres) #genres
                genreslist.append(genres)

                #row[2] homepage removed
                eachnewrow.append(row[3]) #id

                #row[4] original_language removed
                #row[5] original_title removed  
                                            
                #handle overview
                overview = processText(row, 6, 15,overview)

                eachnewrow.append(overview)
                overviewlist.append(overview)

                eachnewrow.append(row[7]) #popularity                  
                
                if row[8] != "":
                    #handle json productioncompanies
                    productioncompanies = readJSON(row, 8, productioncompanies, 'name')

                eachnewrow.append(productioncompanies) #productioncompanies
                productioncompanieslist.append(productioncompanies)                  

                #row[9] productioncountries

                weekday = ""
                day = ""
                month = ""
                year = ""                 

                #handle releasedate 
                if row[10] != "":
                    d = datetime.datetime.strptime(row[10] ,'%Y-%m-%d')

                    weekdayword = d.strftime('%a')
                    weekday = time.strptime(weekdayword, '%a').tm_wday + 1

                    day = d.strftime('%d')

                    monthword = d.strftime('%b')
                    month = time.strptime(monthword,'%b').tm_mon

                    year = d.strftime('%Y')

                eachnewrow.append(weekday) #weekday
                eachnewrow.append(day) #day 
                eachnewrow.append(month) #month
                eachnewrow.append(year) #year

                eachnewrow.append(row[11]) #revenue
                eachnewrow.append(row[12]) #runtime
                #row[13] spoken_languages removed  
                eachnewrow.append(row[14]) #status                      
                #row[15] tagline added to overview
                eachnewrow.append(row[16]) #title
                eachnewrow.append(row[17]) #vote_average
                eachnewrow.append(row[18]) #vote_count                  

                WR = (float(row[18]) / (float(row[18])+25000)) * float(row[17]) + (25000 / (float(row[18]) + 25000)) * 7.0            

                eachnewrow.append(WR) #weighted_rating

                rowslist.append(eachnewrow)

                line_count += 1
        except Exception as e:
            print(f'Row: {line_count} has Exception' + str(e))    
            line_count += 1          
    print(f'Processed {line_count} lines.')   
                  
genresdictionary = corpora.Dictionary(genreslist)
token_to_id_genres = genresdictionary.token2id                

overviewdictionary = corpora.Dictionary(overviewlist)
token_to_id_overview = overviewdictionary.token2id                            
                      
productioncompaniesdictionary = corpora.Dictionary(productioncompanieslist)
token_to_id_productioncompanies = productioncompaniesdictionary.token2id                  
                      
def assigningValues(vector, listofzeros):
    for counter, value in enumerate(vector): 
        index = 0       
        for value2 in listofzeros:                      
            if index == value[0]:
                listofzeros[index] = value[1]    
            index += 1
                  
    return listofzeros

tablecolumns = ['budget'] + list(genresdictionary.values()) + ['id'] + list(overviewdictionary.values()) + ['popularity'] + list(productioncompaniesdictionary.values()) + ['weekday', 'day', 'month', 'year', 'revenue', 'runtime', 'status', 'title', 'vote_average', 'vote_count', 'weighted_rating']

tablerows = []
for eachrow in rowslist:
    listofzerosgenres = [0] * len(token_to_id_genres)
    listofzerosoverview = [0] * len(token_to_id_overview)                          
    listofzerosproductioncompanies = [0] * len(token_to_id_productioncompanies)                    

    vec_genres = genresdictionary.doc2bow(eachrow[1])
    vec_overview = overviewdictionary.doc2bow(eachrow[3])                         
    vec_productioncompanies = productioncompaniesdictionary.doc2bow(eachrow[5])                     

    listofzerosgenres = assigningValues(vec_genres, listofzerosgenres)
    listofzerosoverview = assigningValues(vec_overview, listofzerosoverview)                      
    listofzerosproductioncompanies = assigningValues(vec_productioncompanies, listofzerosproductioncompanies)                      
    
    tablerows.append(eachrow[0:1] + listofzerosgenres + eachrow[2:3] + listofzerosoverview + eachrow[4:5] 
                     + listofzerosproductioncompanies + eachrow[6:17])

df = pd.DataFrame(np.array(tablerows), columns=tablecolumns)                      
print(df.head(10))      
#write to file
""""
with open('processed_tmdb_movies.csv', mode='w', newline='', encoding='utf-8') as movies_file: #change accordingly
    writer2 = csv.writer(movies_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    writer2.writerow(['budget'] + list(genresdictionary.values()) + ['id'] + list(overviewdictionary.values()) + ['popularity'] 
        + list(productioncompaniesdictionary.values()) 
        + ['weekday', 'day', 'month', 'year', 'revenue', 'runtime', 'status', 'title', 'vote_average', 'vote_count', 'weighted_rating'])              
    
    for eachrow in rowslist:
        listofzerosgenres = [0] * len(token_to_id_genres)
        listofzerosoverview = [0] * len(token_to_id_overview)                          
        listofzerosproductioncompanies = [0] * len(token_to_id_productioncompanies)                    
                  
        vec_genres = genresdictionary.doc2bow(eachrow[1])
        vec_overview = overviewdictionary.doc2bow(eachrow[3])                         
        vec_productioncompanies = productioncompaniesdictionary.doc2bow(eachrow[5])                     

        listofzerosgenres = assigningValues(vec_genres, listofzerosgenres)
        listofzerosoverview = assigningValues(vec_overview, listofzerosoverview)                      
        listofzerosproductioncompanies = assigningValues(vec_productioncompanies, listofzerosproductioncompanies)

        writer2.writerow(eachrow[0:1] + listofzerosgenres + eachrow[2:3] + listofzerosoverview + eachrow[4:5] 
        + listofzerosproductioncompanies + eachrow[6:17])
"""
print('\nOutput Success!')                 

Column names are budget, genres, homepage, id, original_language, original_title, overview, popularity, production_companies, production_countries, release_date, revenue, runtime, spoken_languages, status, tagline, title, vote_average, vote_count
Processed 5 lines.
  budget comedy horror documentary      id attack casey charismat defend dojo  \
0      0      1      0           0  480001      1     1         1      1    1   
1      0      0      1           0  480002      0     0         0      0    0   
2      0      0      0           0  480003      0     0         0      0    0   
3      0      0      0           1  480004      0     0         0      0    0   

        ...       day month  year revenue runtime           status  \
0       ...        10     3  2019       0     104  Post Production   
1       ...        01    10  2017       0      46         Released   
2       ...        14    10  2008       0      20         Released   
3       ...        14     3  2008       0      2

<h1>Separating JSON Data into Columns (Credits)</h1>

In [37]:
import csv, json, gensim
from gensim import corpora

rowslist = []
castslist = []
crewslist = []

#read file
with open('tmdb_All_credits.csv', encoding='utf-8') as csv_file: #change accordingly
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        try:
            if line_count == 0:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            elif line_count != 5:
                eachnewrow = []
                casts = []
                crews = []

                eachnewrow.append(row[0]) #id

                #handle json casts
                if row[1] != "":                      
                    jsonstring = row[1]

                    jsonobj = json.loads(jsonstring)

                    for jsonelement in jsonobj:
                        cast = jsonelement['name']       
                        casts.append(cast.lower())

                eachnewrow.append(casts) #casts
                castslist.append(casts)

                #handle json crews 
                if row[2] != "":                       
                    jsonstring2 = row[2]

                    jsonobj2 = json.loads(jsonstring2)

                    for jsonelement in jsonobj2:
                        job = jsonelement['job']
                        if job == "Director":
                            crew = jsonelement['name']       
                            crews.append(crew.lower())

                eachnewrow.append(crews) #crews
                crewslist.append(crews)         

                rowslist.append(eachnewrow)

                line_count += 1
        except Exception as e:
            print(f'Row: {line_count} has Exception' + str(e))    
            line_count += 1   

    print(f'Processed {line_count} lines.')
                  
castsdictionary = corpora.Dictionary(castslist)
token_to_id_casts = castsdictionary.token2id
                  
crewsdictionary = corpora.Dictionary(crewslist)
token_to_id_crews = crewsdictionary.token2id                  

def assigningValues(vector, listofzeros):
    for counter, value in enumerate(vector): 
        index = 0       
        for value2 in listofzeros:                      
            if index == value[0]:
                listofzeros[index] = value[1]    
            index += 1
                  
    return listofzeros                  

tablecolumns2 = ['id'] + list(castsdictionary.values()) + list(crewsdictionary.values())
                      
tablerows2 = []        
                      
for eachrow in rowslist:
    listofzeroscasts = [0] * len(token_to_id_casts)
    listofzeroscrews = [0] * len(token_to_id_crews)                         

    vec_casts = castsdictionary.doc2bow(eachrow[1])
    vec_crews = crewsdictionary.doc2bow(eachrow[2])   

    listofzeroscasts = assigningValues(vec_casts, listofzeroscasts)
    listofzeroscrews = assigningValues(vec_crews, listofzeroscrews)

    tablerows2.append(eachrow[0:1] + listofzeroscasts + listofzeroscrews)
                      
df2 = pd.DataFrame(np.array(tablerows2), columns=tablecolumns2)                      
print(df2.head(10))                        
"""                     
#write to file
with open('processed_tmdb_credits.csv', mode='w', newline='', encoding='utf-8') as credits_file: #change accordingly
    writer = csv.writer(credits_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    writer.writerow(['id'] + list(castsdictionary.values()) + list(crewsdictionary.values()))              
    
    for eachrow in rowslist:
        listofzeroscasts = [0] * len(token_to_id_casts)
        listofzeroscrews = [0] * len(token_to_id_crews)                         
                  
        vec_casts = castsdictionary.doc2bow(eachrow[1])
        vec_crews = crewsdictionary.doc2bow(eachrow[2])   

        listofzeroscasts = assigningValues(vec_casts, listofzeroscasts)
        listofzeroscrews = assigningValues(vec_crews, listofzeroscrews)

        writer.writerow(eachrow[0:1] + listofzeroscasts + listofzeroscrews)
"""
print('\nOutput Success!')                  

Column names are id, cast, crew
Processed 5 lines.
       id alessandro nivola david zellner imogen poots jason burkey  \
0  480001                 1             1            1            1   
1  480002                 0             0            0            0   
2  480003                 0             0            0            0   
3  480004                 0             0            0            0   

  jesse eisenberg phillip andre botello steve terada george bonilla  \
0               1                     1            1              0   
1               0                     0            0              1   
2               0                     0            0              0   
3               0                     0            0              0   

  bruce scivally    ...    robert yanal rudy behlmer scott mcisaac  \
0              0    ...               0            0             0   
1              0    ...               0            0             0   
2              0    ...    

<h1>Merging Two Files</h1>

In [40]:
import pandas as pd

df3 = df.join(df2.set_index('id'), on='id')
print(df3.head(10)) 
"""
movies = pd.read_csv("processed_tmdb_movies.csv") #change accordingly
credits = pd.read_csv("processed_tmdb_credits.csv") #change accordingly
merged = pd.merge(movies, credits, left_on=['id'],
                   right_on=['id'], 
                   how='left')
merged.to_csv("processed_tmdb_data.csv", index=False)
"""
print('\nOutput Success!')  

  budget comedy horror documentary      id attack casey charismat defend dojo  \
0      0      1      0           0  480001      1     1         1      1    1   
1      0      0      1           0  480002      0     0         0      0    0   
2      0      0      0           0  480003      0     0         0      0    0   
3      0      0      0           1  480004      0     0         0      0    0   

     ...    robert yanal rudy behlmer scott mcisaac shad meshad  \
0    ...               0            0             0           0   
1    ...               0            0             0           0   
2    ...               0            0             0           0   
3    ...               1            1             1           1   

  stephen rebello thomas schatz riley stearns george bonilla lisa van eyssen  \
0               0             0             1              0               0   
1               0             0             0              1               0   
2               0 

<h1>Append New Processed Data to Previous Merged File</h1>

In [36]:
import csv

with open('main_processed_tmdb_data.csv', mode='a', newline='', encoding='utf-8') as main_file:
    writer3 = csv.writer(main_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    with open('processed_tmdb_data.csv', mode='r', newline='') as new_file:
        csv_reader = csv.reader(new_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                line_count += 1
            else:
                writer3.writerow(row)
                line_count += 1
            
    print(f'Processed {line_count} lines.')
    
    print('\nOutput Success!')  

Processed 4804 lines.

Output Success!
